In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rcParams['figure.facecolor']=(1,1,1,1) # pycharm 绘图白底，看得清坐标
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

train = pd.read_csv("./train_set.csv")
test = pd.read_csv("./test_set.csv")

In [74]:
# 将训练集拆分一些出来做验证, 分层抽样
from sklearn.model_selection import StratifiedShuffleSplit
splt = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
for train_idx, vaild_idx in splt.split(train,train['y']):
    train_part = train.loc[train_idx]
    valid_part = train.loc[vaild_idx]
    
train_part_y = train_part['y']
valid_part_y = valid_part['y']
y_train = train['y']

In [75]:
train['default_']=0
train['default_'][train['default']=='yes'] = 1
train['housing_']=0
train['housing_'][train['housing']=='yes'] = 1
train['loan_']=0
train['loan_'][train['loan']=='yes'] = 1

test['default_']=0
test['default_'][test['default']=='yes'] = 1
test['housing_']=0
test['housing_'][test['housing']=='yes'] = 1
test['loan_']=0
test['loan_'][test['loan']=='yes'] = 1

train_part['default_']=0
train_part['default_'][train_part['default']=='yes'] = 1
train_part['housing_']=0
train_part['housing_'][train_part['housing']=='yes'] = 1
train_part['loan_']=0
train_part['loan_'][train_part['loan']=='yes'] = 1

valid_part['default_']=0
valid_part['default_'][valid_part['default']=='yes'] = 1
valid_part['housing_']=0
valid_part['housing_'][valid_part['housing']=='yes'] = 1
valid_part['loan_']=0
valid_part['loan_'][valid_part['loan']=='yes'] = 1

<ipython-input-75-34e6b5587db4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['default_'][train['default']=='yes'] = 1
<ipython-input-75-34e6b5587db4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['housing_'][train['housing']=='yes'] = 1
<ipython-input-75-34e6b5587db4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['loan_'][train['loan']=='yes'] = 1
<ipython-input-75-34e6b5587db4>:9: SettingWithCopyWarning: 
A

In [78]:
X_train = train.drop(['y','ID','default','housing','loan'], axis=1)
X_test = test.drop(['ID','default','housing','loan'], axis=1)

train_part = train_part.drop(['y','ID','default','housing','loan'], axis=1)
valid_part = valid_part.drop(['y','ID','default','housing','loan'], axis=1)

def num_cat_splitor(X_train):
    s = (X_train.dtypes == 'object')
    object_cols = list(s[s].index)
    num_cols = list(set(X_train.columns) - set(object_cols))
    return num_cols, object_cols
num_cols, object_cols = num_cat_splitor(X_train)
num_cols
object_cols

enc = OneHotEncoder(sparse=False,handle_unknown='ignore')
enc.fit_transform(X_train[object_cols])
arr = enc.transform(X_test[object_cols])
arr

KeyError: "['y' 'ID' 'default' 'housing' 'loan'] not found in axis"

In [87]:
fn = enc.get_feature_names(object_cols)
print(fn)
test_oh = pd.DataFrame(arr)
test_oh.columns = fn
test_oh['ID'] = test['ID']
test_oh.to_csv('test_oh.csv')

['job_admin.' 'job_blue-collar' 'job_entrepreneur' 'job_housemaid'
 'job_management' 'job_retired' 'job_self-employed' 'job_services'
 'job_student' 'job_technician' 'job_unemployed' 'job_unknown'
 'marital_divorced' 'marital_married' 'marital_single' 'education_primary'
 'education_secondary' 'education_tertiary' 'education_unknown'
 'contact_cellular' 'contact_telephone' 'contact_unknown' 'month_apr'
 'month_aug' 'month_dec' 'month_feb' 'month_jan' 'month_jul' 'month_jun'
 'month_mar' 'month_may' 'month_nov' 'month_oct' 'month_sep'
 'poutcome_failure' 'poutcome_other' 'poutcome_success' 'poutcome_unknown']


In [65]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
        
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_cols)),
#         ('imputer', SimpleImputer(strategy="median")),
#         ('std_scaler', StandardScaler()),
    ])
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(object_cols)),
        ('cat_encoder', OneHotEncoder(sparse=False,handle_unknown='ignore')),
    ])
full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

print(X_train.head(5))
X_prepared = full_pipeline.fit_transform(X_train)
print(X_prepared)
X_train.head(5)

   age         job   marital  education  balance   contact  day month  \
0   43  management   married   tertiary      291   unknown    9   may   
1   42  technician  divorced    primary     5076  cellular    7   apr   
2   47      admin.   married  secondary      104  cellular   14   jul   
3   28  management    single  secondary     -994  cellular   18   jul   
4   42  technician  divorced  secondary     2974   unknown   21   may   

   duration  campaign  pdays  previous poutcome  default_  housing_  loan_  
0       150         2     -1         0  unknown         0         1      0  
1        99         1    251         2    other         0         1      0  
2        77         2     -1         0  unknown         0         1      1  
3       174         2     -1         0  unknown         0         1      1  
4       187         5     -1         0  unknown         0         1      0  
[[1.500e+02 9.000e+00 4.300e+01 ... 0.000e+00 0.000e+00 1.000e+00]
 [9.900e+01 7.000e+00 4.200e+01 

,age,job,marital,education,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,default_,housing_,loan_
0,43,management,married,tertiary,291,unknown,9,may,150,2,-1,0,unknown,0,1,0
1,42,technician,divorced,primary,5076,cellular,7,apr,99,1,251,2,other,0,1,0
2,47,admin.,married,secondary,104,cellular,14,jul,77,2,-1,0,unknown,0,1,1
3,28,management,single,secondary,-994,cellular,18,jul,174,2,-1,0,unknown,0,1,1
4,42,technician,divorced,secondary,2974,unknown,21,may,187,5,-1,0,unknown,0,1,0


In [66]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

perceptron = Perceptron()
rf = RandomForestClassifier(n_estimators=300)
knn = KNeighborsClassifier()
lr = LogisticRegression()
svc = SVC()
gbdt = GradientBoostingClassifier(learning_rate=0.1, n_estimators=130, max_features='sqrt', max_depth=5, min_samples_split=1200, min_samples_leaf=60, subsample=0.8, random_state=10)

# models = [perceptron, rf, knn, lr, svc]
models = [rf, gbdt]

for model in models:
    prepare_select_and_predict_pipeline = Pipeline([
        ('preparation', full_pipeline),
        ('model', model)
    ])
    prepare_select_and_predict_pipeline.fit(train_part,train_part_y)
    pred = prepare_select_and_predict_pipeline.predict(valid_part)
    print(model)
    print("auc score: ", roc_auc_score(valid_part_y, pred))

    # param_grid = [{
    # #     'preparation__num_pipeline__imputer__strategy': ['mean', 'median', 'most_frequent'],
    #     'model__n_estimators' : [250,300,330,350,400,500],
    #     'model__max_features':[5,10,20,30,50]
    # }]

    # grid_search_prep = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=3,
    #                                 scoring='roc_auc', verbose=2, n_jobs=-1)
    # grid_search_prep.fit(train_part, train_part_y)
    # grid_search_prep.best_params_
    # final_model = grid_search_prep.best_estimator_
    # y_pred_test = final_model.predict(valid_part)
    # print("auc score: ", roc_auc_score(valid_part_y, y_pred_test))
    #
#     result = pd.DataFrame()
#     result['ID'] = test['ID']
#     result['pred'] = y_pred_test
#     result.to_csv('buy_product_pred.csv',index=False)

RandomForestClassifier(n_estimators=300)
auc score:  0.6945770681235798


KeyboardInterrupt: 